In [1]:
import json

import kfp
from kfp import dsl
import os
import kfp.compiler as compiler
from kubernetes.client.models import V1EnvVar
from kubernetes import client as k8s_client

# List existing pipeline experiments

In [2]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)

In [3]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2021, 9, 5, 13, 34, 40, tzinfo=tzlocal()),
                  'description': None,
                  'id': '609fee28-4a55-414e-90a4-456591d1de63',
                  'name': 'Default',
                  'resource_references': [{'key': {'id': 'ocdkube',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'},
                 {'created_at': datetime.datetime(2021, 9, 6, 14, 21, 13, tzinfo=tzlocal()),
                  'description': None,
                  'id': 'aab4d596-ffa7-4c54-bcb7-0667105b1ca9',
                  'name': 'Dkube - Storage pl',
                  'resource_references': [{'key': {'id': 'ocdkube',
                                                   'type': 'NAMESPACE'},
                                           'name': N

# Create a Dkube Storage experiment

In [4]:
dkube_store_experiment = client.create_experiment(name='Dkube - Storage pl')

In [5]:
class ContainerOp(kfp.dsl.ContainerOp):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.add_pod_label(name="dkube.garbagecollect", value="true")
        self.add_pod_label(name="dkube.garbagecollect.policy", value="all")
        self.add_pod_label(name="runid", value="{{pod.name}}")
        self.add_pod_label(name="wfid", value="{{workflow.uid}}")

def reclaim(uid, namespace, auth_token):
    return ContainerOp(
       name="reclaim_repos",
       image="ocdr/dkubepl:2.3.0.6",
       command=[
           "dkubepl",
           "storage",
           "--action",
           "reclaim",
           "--token",
           auth_token,
           "--namespace",
           namespace,
           "--uid",
           uid
       ],
    )
    
    
        
@kfp.dsl.pipeline(
    name="MNIST Storage pipeline",
    description="A pipline showing how to use storage component",
)
def storage_pipeline(
    #Dkube authentication token
    auth_token = os.getenv("DKUBE_USER_ACCESS_TOKEN"),
    #Namespace to export the datasets
    namespace = os.getenv("USERNAME"),
    #List of artifacts to export, format : <claimname>@<class>://<owner>:<datumname>/<version>
    #Class must be one of: [dataset, model, program]
    #To get the latest version use "latest" in place of <version>, otherwise use v1, v2, etc. (For program, use full commitid)
    #(Default version is "latest")
    dataset = "mnist/v1",
    program = "mnist",
    model = "mnist"
    ):
    
    reclaim_task = reclaim("{{workflow.uid}}", namespace, auth_token) 
    with dsl.ExitHandler(reclaim_task):
        program_volume_claim = "{{workflow.uid}}-mn-project"
        dataset_volume_claim = "{{workflow.uid}}-mn-dataset"
        model_volume_claim = "{{workflow.uid}}-mn-model"
        program_volume = program_volume_claim + "@program://" + str(program)
        dataset_volume = dataset_volume_claim + "@dataset://" + str(dataset)
        model_volume = model_volume_claim + "@model://" + str(model)
        input_volumes = json.dumps([program_volume, dataset_volume])
        output_volumes = json.dumps([model_volume])
        
        pipelineConfig = kfp.dsl.PipelineConf()
        pipelineConfig.set_image_pull_policy("Always")
        
        export = ContainerOp(
            name="export_repos",
            image="ocdr/dkubepl:2.3.0.6",
            command=[
                "dkubepl",
                "storage",
                "--action",
                "export",
                "--token",
                auth_token,
                "--namespace",
                namespace,
                "--input_volumes",
                input_volumes,
                "--output_volumes",
                output_volumes,
                "--runid",
                '{{pod.name}}',
                "--wfid",
                "{{workflow.uid}}",
            ],
        )
    
        train = ContainerOp(name='train', image='ocdr/d3-datascience-tf-cpu:v1.14', command="bash", arguments=["-c", "cd /opt/dkube/project/; python mnist/train.py"]).add_volume(k8s_client.V1Volume(name='mnist-project', persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name=program_volume_claim))).add_volume_mount(k8s_client.V1VolumeMount(mount_path='/opt/dkube/project', name='mnist-project')).add_volume(k8s_client.V1Volume(name='mnist-dataset', persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name=dataset_volume_claim))).add_volume_mount(k8s_client.V1VolumeMount(mount_path='/mnist', name='mnist-dataset')).add_volume(k8s_client.V1Volume(name='mnist-model', persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name=model_volume_claim))).	add_volume_mount(k8s_client.V1VolumeMount(mount_path='/model', name='mnist-model'))
       # train = ContainerOp(name='train', image='ocdr/d3-datascience-tf-cpu:v1.14', command="bash", arguments=["-c", "sleep infinity"]).add_volume(k8s_client.V1Volume(name='mnist-project', persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name=program_volume_claim))).add_volume_mount(k8s_client.V1VolumeMount(mount_path='/opt/dkube/project', name='mnist-project')).add_volume(k8s_client.V1Volume(name='mnist-dataset', persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name=dataset_volume_claim))).add_volume_mount(k8s_client.V1VolumeMount(mount_path='/opt/dkube/input', name='mnist-dataset')).add_volume(k8s_client.V1Volume(name='mnist-model', persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name=model_volume_claim))).	add_volume_mount(k8s_client.V1VolumeMount(mount_path='/opt/dkube/output', name='mnist-model'))
        #train.add_pod_label(name: str, value: str)

        train.after(export)
   


In [6]:
compiler.Compiler().compile(storage_pipeline, 'dkube_storage_pl.tar.gz')

/usr/local/lib/python3.6/dist-packages/kfp/dsl/_container_op.py:1157: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


In [7]:
run = client.run_pipeline(dkube_store_experiment.id, 'dkube_storage_pipeline_1', 'dkube_storage_pl.tar.gz', params={})